# Compare translations

Two works are compared using frequencies

In [5]:
import dhlab as dh

import dhlab.module_update as mu
import pandas as pd
import re

from werkzeug.utils import secure_filename

import dhlab.nbtext as nb
import warnings
warnings.filterwarnings("ignore")

In [6]:
mu.css()

In [7]:
titles = lambda bb: bb.groupby('title').count().urn.sort_values(ascending = False)

In [8]:
#!pip install distance

import distance as dist

def propose_corr(candidate_dict, cutoff = 3):
    """ Try levenshtein for the candidate words"""
    res = list()
    for x in candidate_dict:
        xl = list()
        for y in candidate_dict[x]:
            d = dist.levenshtein(x.lower(), y.lower())
            if d <= cutoff:
                xl.append((y, d))
        xl.sort(key = lambda x:x[1])
        if len(xl) > 0:
            min_value = xl[0][1]
            #print(min_value)
            candidates = [(x,w[0],w[1]) for w in xl if w[1] == min_value] 
            the_candidate = candidates[0]
            if len(candidates) > 1:
                # heuristic so that if everything else is equal, choose the one with same first letter
                # might create problems for aand ånd ...
                smallset = [(x, w, s) for (x, w, s) in candidates if x.lower()[0] == w.lower()[0]]
                if smallset != []:
                    the_candidate = smallset[0]
            res.append(the_candidate)
    return res

In [11]:
def make_candidates(lang_text = None, trans_text = None, diff_window = 0.05, cutoff_sim = 20):
    """Compute word equivalents between old_text and new_text. They are assumed to be two versions of the same text"""
    
    check = lambda w: combo[(combo['orig']/combo.loc[w]['trans']) < 1 + diff_window][1 - diff_window < (combo['orig']/combo.loc[w]['trans'])]
    
    # get frequencies and make dataframes
    ag = dh.Counts(old_text).counts
    an = dh.Counts(new_text).counts

    combo = pd.concat([ag, an], axis = 1).fillna(0)

    combo['sim'] = combo.ag/combo.an
    
    # candidate words are grabbed from words that have a similar frequency across two languages
    subs = combo.loc[combo[combo['sim']] <= cutoff_diff]    
    # search using function check() for words with similar frequency
    candidate_dict = {x:check(x).index for x in subs.index}
    
    return candidate_dict

In [12]:
def make_candidates_from_agg(orig_id = None, trans_id = None, diff_window = 0.05, cutoff = 10):
    """Compute word equivalents between text and its translation."""
    
    check = lambda w: combo[(combo['orig']/combo.loc[w]['trans']) < 1 + diff_window][1 - diff_window < (combo['orig']/combo.loc[w]['trans'])]
    
    # get frequencies and make dataframes

    orig_text = dh.Counts(orig_id).counts
    trans_text = dh.Counts(trans_id).counts

    combo = pd.concat([orig_text, trans_text], axis = 1).fillna(0)
    combo.columns = ["orig", "trans"]
    combo['sim'] = combo.orig/combo.combo.trans
    
    # candidate words are grabbed from words that have a sizable different frequency across the two versions
    subs = combo.loc[combo[combo['orig'] <= cutoff][combo['trans'] >= cutoff].index]
    
    # search using function check() for words with similar frequency
    candidate_dict = {x:check(x).index for x in subs.index}
    
    return candidate_dict

## Bokpar

Bokpar letes frem med metadata. Titler som har mer enn en oppføring, sjekkes for skrivemåter med bruk av konkordanser, for eksempel "får" vs "faar". Teksten som inneholder siste skrivemåte er den eldre. Konkordansene gir også et inntrykk av OCR-kvaliteten.

### Forfatter

Variabelen `forfatter` brukes til fil-lagring nedenfor. Her skal vi bruke korpus mot korpus. For Ibsen gjelder det samlede verker fra tidlig 1900-tall mot et moderne fra 2000.

### Ordkoblinger

Beregn kobling mellom ord, gammel til ny, og legg resultatet i en dataramme.

Parametrene for make_candidates_from_agg er to aggregerte korpus. Deretter et krav om forskjell, og deretter en cutoff på frekvens. Lavfrekvente ord utgjør over halvparten av all tekst, og vil redusere presisjonen om alt tas med. En grense på 3 eller 4 virker ok.